<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/ResNet152.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 30.5 MB/s 
     |████████████████████████████████| 182 kB 72.2 MB/s 
     |████████████████████████████████| 162 kB 68.1 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 162 kB 78.4 MB/s 
     |████████████████████████████████| 158 kB 74.4 MB/s 
     |████████████████████████████████| 157 kB 73.8 MB/s 
     |████████████████████████████████| 157 kB 79.3 MB/s 
     |████████████████████████████████| 157 kB 66.3 MB/s 
     |████████████████████████████████| 157 kB 75.6 MB/s 
     |████████████████████████████████| 157 kB 79.8 MB/s 
     |████████████████████████████████| 157 kB 81.8 MB/s 
     |████████████████████████████████| 157 kB 75.8 MB/s 
     |████████████████████████████████| 156 kB 62.3 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wand

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [ ]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [ ]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbign and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [ ]:
resnet152 = tf.keras.applications.ResNet152(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None)
model_resnet152=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet152,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet152.trainable=False
model_resnet152.summary()

234708992/234698864 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet152 (Functional)      (None, None, None, 2048)  58370944  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1028)              103162884 
                                                                 
 dense_1 (Dense)             (None, 5)                 5145      
                                                                 
Total params: 161,538,973
Trainable params: 103,168,029
Non-trainable params: 58,370,944
______________________________

In [ ]:
model_resnet152.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet152 with hidden dense",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [ ]:
%%time
epochs=40
history = model_resnet152.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 18.5271 - accuracy: 0.3649

wandb: Adding directory to artifact (/content/wandb/run-20221012_092338-2e2tf4vg/files/model-best)... Done. 8.7s


19/19 [==============================] - 267s 9s/step - loss: 18.5271 - accuracy: 0.3649 - val_loss: 2.9054 - val_accuracy: 0.6081
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 1.9686 - accuracy: 0.5000

wandb: Adding directory to artifact (/content/wandb/run-20221012_092338-2e2tf4vg/files/model-best)... Done. 8.7s


...

19/19 [==============================] - 93s 5s/step - loss: 0.3469 - accuracy: 0.9071 - val_loss: 0.4603 - val_accuracy: 0.8514
Epoch 27/40
19/19 [==============================] - 8s 373ms/step - loss: 0.3127 - accuracy: 0.9139 - val_loss: 0.5234 - val_accuracy: 0.7905
Epoch 28/40
19/19 [==============================] - 8s 375ms/step - loss: 0.3379 - accuracy: 0.9003 - val_loss: 0.5378 - val_accuracy: 0.7703
Epoch 29/40
19/19 [==============================] - 8s 380ms/step - loss: 0.3956 - accuracy: 0.8564 - val_loss: 0.5077 - val_accuracy: 0.8446
Epoch 30/40
19/19 [==============================] - ETA: 0s - loss: 0.3632 - accuracy: 0.8733

wandb: Adding directory to artifact (/content/wandb/run-20221012_092338-2e2tf4vg/files/model-best)... Done. 8.8s


19/19 [==============================] - 86s 5s/step - loss: 0.3632 - accuracy: 0.8733 - val_loss: 0.4564 - val_accuracy: 0.8851
Epoch 31/40
19/19 [==============================] - 8s 373ms/step - loss: 0.5424 - accuracy: 0.8074 - val_loss: 0.6070 - val_accuracy: 0.7838
Epoch 32/40
19/19 [==============================] - 8s 375ms/step - loss: 0.4462 - accuracy: 0.8142 - val_loss: 0.6779 - val_accuracy: 0.7162
Epoch 33/40
19/19 [==============================] - 8s 384ms/step - loss: 0.4677 - accuracy: 0.8108 - val_loss: 0.8557 - val_accuracy: 0.6757
Epoch 34/40
19/19 [==============================] - 8s 381ms/step - loss: 0.4891 - accuracy: 0.8159 - val_loss: 0.5125 - val_accuracy: 0.8446
Epoch 35/40
19/19 [==============================] - 8s 375ms/step - loss: 0.2610 - accuracy: 0.9358 - val_loss: 0.4873 - val_accuracy: 0.8108
Epoch 36/40
19/19 [==============================] - 8s 374ms/step - loss: 0.2937 - accuracy: 0.9105 - val_loss: 0.5043 - val_accuracy: 0.8176
Epoch 37/40
1

wandb: Adding directory to artifact (/content/wandb/run-20221012_092338-2e2tf4vg/files/model-best)... Done. 8.8s


19/19 [==============================] - 85s 5s/step - loss: 0.2892 - accuracy: 0.9037 - val_loss: 0.4520 - val_accuracy: 0.8176
Epoch 39/40
19/19 [==============================] - 8s 381ms/step - loss: 0.3509 - accuracy: 0.8902 - val_loss: 0.6055 - val_accuracy: 0.7770
Epoch 40/40
19/19 [==============================] - 8s 375ms/step - loss: 0.2661 - accuracy: 0.9088 - val_loss: 0.6677 - val_accuracy: 0.7432
CPU times: user 16min 29s, sys: 1min 59s, total: 18min 28s
Wall time: 32min 43s


In [ ]:
wandb.finish()

accuracy,▁▃▃▃▄▅▅▅▆▆▆▆▄▅▆▇▇▇▆▇▆▇▇▇▇███▇▇▆▇▆▇████▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▃▁▁▄▄▄▅▅▅▅▅▂▃▆▇▆▇▆▆▆▆▇▇▆▆▇▆▆▇█▆▅▅▇▇▇▇▇▆▆
val_loss,█▄▆▃▂▂▂▂▂▂▂▃▃▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▂
accuracy,0.90878
best_epoch,37
best_val_loss,0.45199
epoch,39
loss,0.26607
val_accuracy,0.74324


In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv